In [4]:
import storage as storage
import os
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="seagull12",
  database="forestry"
)

mycursor = mydb.cursor()

def getCountries(top_three):
    country1 = None
    country2 = None
    country3 = None
    if len(top_three) == 1:
        country1 = top_three[0]
    if len(top_three) == 2:
        country2 = top_three[1]
    elif len(top_three) == 3:
        country3 = top_three[2]
        
    return country1, country2, country3

def getFirst(lst):
    result = None
    if len(lst) > 0:
        result = lst[0]
    return result

def insertPaper(paper, country):
    sql_insert = "INSERT INTO PolicyPapers " \
        "(filename, country, country1, country2, country3, " \
        "date, title) " \
        "VALUES (%s, %s, %s, %s, %s, %s, %s)"

    country1, country2, country3 = getCountries(paper.country[:3])

    val = (paper.filename, country, country1, country2, country3, getFirst(paper.date),
        "")
    mycursor.execute(sql_insert, val)
    
    sql_insert = "INSERT INTO PolicyPaperPages " \
        "(filename, pid, text) " \
        "VALUES (%s, %s, %s)"
    for i in range(len(paper.extractedText)):
        val = (paper.filename, i + 1, paper.extractedText[i])
        mycursor.execute(sql_insert, val)

    mydb.commit()

def inDatabase(filename):
    mycursor.execute("SELECT filename FROM PolicyPapers WHERE filename='%s'" % filename)
    return mycursor.fetchone()

In [5]:
def getCountryName(file_path):
    return file_path.rsplit('/', 2)[1]

# Might need to generalize multiple levels of directories
def extractPolicyPapersFromDirectory(directory_path):    
    # Retrieve file names inside folder
    file_path_errors = []
    for subdir, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = subdir + os.sep + file
            
            if file == ".DS_Store":
                continue

            if not inDatabase(storage.getFileName(file_path)):
                print("PROCESSING", file_path)
                try:
                    paper = storage.PolicyPaper(file_path)
                    country = getCountryName(subdir)
                    insertPaper(paper, country)
                except:
                    file_path_errors.append(file_path)
                    print("ERROR: Could not convert", file_path)
                    
    print("EXTRACTION COMPLETE")
    print()
    
    if len(file_path_errors) > 0:
        print("The following files were not successfully processed:")
        for file_path in file_path_errors:
            print(file_path)
    
extractPolicyPapersFromDirectory("Policy PDFs")

PROCESSING Policy PDFs/India/THE ENVIRONMENT (PROTECTION) ACT, 1986.pdf
Paper successfully converted!
['THE', 'ENVIRONMENT', '(PROTECTION)', 'ACT,', '1986No.', '29', 'OF', '1986[23rd', 'May,', '1986.]An', 'Act', 'to', 'provide', 'for', 'the', 'protection', 'and', 'improvement', 'of', 'environment', 'and', 'for', 'matters', 'connected', 'there', 'with:WHEREAS', 'the', 'decisions', 'were', 'taken', 'at', 'the', 'United', 'NationsConference', 'on', 'the', 'Human', 'Environment', 'held', 'atStockholm', 'in', 'June,', '1972,', 'in', 'which', 'India', 'participated,', 'to', 'take', 'appropriate', 'steps', 'for', 'the', 'protection', 'andimprovement', 'of', 'human', 'environment;AND', 'WHEREAS', 'it', 'is', 'considered', 'necessary', 'further', 'to', 'implement', 'the', 'decisions', 'aforesaid', 'in', 'so', 'far', 'as', 'theyrelate', 'to', 'the', 'protection', 'and', 'improvement', 'of', 'environment', 'and', 'the', 'prevention', 'of', 'hazards', 'to', 'human', 'beings,other', 'living', 'crea

Paper successfully converted!
['The', 'Circular', 'Concerning', 'Joint', 'Forest', 'Management', 'No.', '6-21/89-P.P', 'Government', 'of', 'India', 'Ministry', 'of', 'Environment', 'and', 'Forests', 'Department', 'of', 'Environment,', 'Forests', 'and', 'Wildlife', 'Paryavaran', 'Bhavan,', 'C.G.O.', 'Complex,', 'B-Block', 'Lodi', 'Road,', 'New', 'Delhi', 'Dated:', '1st', 'June,', '1990', 'The', 'Forest', 'Secretaries', '(All', 'States/UTs)', 'Subject:', 'Involving', 'of', 'village', 'communities', 'and', 'voluntary', 'agencies', 'for', 'regeneration', 'of', 'degraded', 'forest', 'lands.', 'Sir,', 'The', 'National', 'Forest', 'Policy,', '1988', 'envisages', "people's", 'involvement', 'in', 'the', 'development', 'and', 'protection', 'of', 'forests.', 'The', 'requirements', 'of', 'fuel-wood,', 'fodder', 'and', 'small', 'timber', 'such', 'as', 'house', 'building', 'material,', 'of', 'the', 'tribals', 'and', 'other', 'villager', 'living', 'in', 'and', 'near', 'the', 'forests,', 'are', 'to', 

Paper successfully converted!
Metadata extracted!
PROCESSING Policy PDFs/India/State Action Plan_Madhya-Pradesh_2014.pdf
Paper successfully converted!
Metadata extracted!
PROCESSING Policy PDFs/India/MineralConcessionRules1960.pdf
Paper successfully converted!
Metadata extracted!
PROCESSING Policy PDFs/India/15. Wildlife (Protection) Act, 1972.pdf
Paper successfully converted!
Metadata extracted!
PROCESSING Policy PDFs/India/NationalGreenTribunalAct2010.pdf
Paper successfully converted!
Metadata extracted!
PROCESSING Policy PDFs/India/State Action Plan_Andhra-pradesh_2012.pdf
Paper successfully converted!
Metadata extracted!
PROCESSING Policy PDFs/India/Biological Diversity Act 2002.pdf
Paper successfully converted!
Metadata extracted!
PROCESSING Policy PDFs/India/ProtectionofPlantVarietiesandFarmersRightsAct2001.pdf
Paper successfully converted!
Metadata extracted!
PROCESSING Policy PDFs/India/THE SCHEDULED TRIBES AND OTHER TRADITIONAL FOREST DWELLERS (RECOGNITION OF FOREST RIGHTS) AC